In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  1 09:47:02 2018
@author: pabloruizruiz
"""

import os
import sys
import pickle
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

from utils import avoidWarnings
from beautifultable import BeautifulTable as BT

avoidWarnings()

In [5]:
''' OPTIMIZER PARAMETERS - Analysis on those '''

best_acc = 0  
start_epoch = 0  
num_epochs = 40  ## TODO: set to args.epochs
batch_size = 128  ## TODO: set to args.barch
milestones = [100, 150]

L = 16
M = 32
E = 5
 
comments = True
n_workers = torch.multiprocessing.cpu_count()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpus = True if torch.cuda.device_count() > 1 else False
device_name = torch.cuda.get_device_name(0) if device == 'cuda' else 'CPUs'
    
table = BT()
table.append_row(['Python Version', sys.version[:5]])
table.append_row(['PyTorch Version', torch.__version__])
table.append_row(['Device', str(device_name)])
table.append_row(['Cores', str(n_workers)])
table.append_row(['GPUs', str(torch.cuda.device_count())])
table.append_row(['CUDNN Enabled', str(torch.backends.cudnn.enabled)])
table.append_row(['Architecture', 'DenseNet x7'])
table.append_row(['Dataset', 'CIFAR10'])
table.append_row(['Epochs', str(num_epochs)])
table.append_row(['Batch Size', str(batch_size)])

print(table)

+-----------------+----------------------+
| Python Version  |        3.6.5         |
+-----------------+----------------------+
| PyTorch Version |        1.0.0         |
+-----------------+----------------------+
|     Device      | Tesla V100-SXM2-16GB |
+-----------------+----------------------+
|      Cores      |          8           |
+-----------------+----------------------+
|      GPUs       |          1           |
+-----------------+----------------------+
|  CUDNN Enabled  |         True         |
+-----------------+----------------------+
|  Architecture   |     DenseNet x7      |
+-----------------+----------------------+
|     Dataset     |       CIFAR10        |
+-----------------+----------------------+
|     Epochs      |          40          |
+-----------------+----------------------+
|   Batch Size    |         128          |
+-----------------+----------------------+


In [6]:
# Data
# ----

avoidWarnings()
dataset = 'MNIST'
dataset = 'CIFAR'
from data import dataloaders
trainloader, testloader, classes = dataloaders(dataset, batch_size)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [8]:
# Models 
# ------
    
# For now, NO SHARING of any layers withing the ensemble

lr = 0.01
avoidWarnings()
comments = False
from models import Conv_Net
from utils import count_parameters
net = Conv_Net('net', layers=L, filters=M)

print('Regular net')
if comments: print(net)
print('\n\n\t\tParameters: {}M'.format(count_parameters(net)/1e6))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)

Regular net


		Parameters: 0.174634M


In [9]:
# Training
# --------
    
from utils import timeit
from results import TrainResults as Results

def train(epoch):
    
    net.train()
    print('\nEpoch: %d' % epoch)

    total = 0
    correct = 0
    train_loss = 0
    global results
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    accuracy = 100.*correct/total        
    print('Train :: Loss: {} | Accy: {}'.format(train_loss, accuracy))

        
def test(epoch):
    
    net.eval()

    total = 0
    correct = 0
    test_loss = 0
    global results
    global best_acc

    with torch.no_grad():
        
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
                    
        accuracy = 100.*correct/total
        print('Valid :: Loss: {} | Accy: {}'.format(test_loss, accuracy))
            
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc


def lr_schedule(epoch):

    global milestones
    if epoch == milestones[0] or epoch == milestones[1]:
        for p in optimizer.param_groups:  p['lr'] = p['lr'] / 10
        print('\n** Changing LR to {} \n'.format(p['lr']))    
    return
    
def results_backup(epoch):
    # Save every X epochs in case training breaks we don't loose results    
    global results
    if epoch % 20 == 0:
        with open('Results_Singe.pkl', 'wb') as object_result:
                pickle.dump(results, object_result, pickle.HIGHEST_PROTOCOL)     


@timeit
def run_epoch(epoch):
    
    lr_schedule(epoch)
    train(epoch)
    test(epoch)
    results_backup(epoch)